In [0]:
# #colabを使う方はこちらを使用ください。
# !pip install torch==0.4.1
# !pip install torchvision==0.2.1
# !pip install numpy==1.14.6
# !pip install matplotlib==2.1.2
# !pip install pillow==5.0.0
# !pip install opencv-python==3.4.3.18

In [0]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

# ネットワーク定義

### Sequentialを使った書き方



In [0]:
model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )
model

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (3): ReLU()
)

In [0]:
model = torch.nn.Sequential()
model.add_module("conv1", nn.Conv2d(1,20,5))
model.add_module("relu1", nn.ReLU())
model.add_module("conv2", nn.Conv2d(20,64,5))
model.add_module("relu2", nn.ReLU())
model

Sequential(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)

In [0]:
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))
model

Sequential(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)

## nn.Moduleを継承した書き方


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 64, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))
      
model = Model()
model

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
)

## nn.ModuleListを使った書き方


In [3]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.convs = nn.ModuleList([nn.Conv2d(1, 20, 5), nn.Conv2d(20, 64, 5)])

    def forward(self, x):
        for i, l in enumerate(self.convs):
            x = l(x)
        return x
      
model = Model()
model

Model(
  (convs): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  )
)

## nn.ModuleDictを使った書き方

In [4]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.convs = nn.ModuleDict({'conv1' : nn.Conv2d(1, 20, 5), 'conv2' : nn.Conv2d(20, 64, 5)})

    def forward(self, x):
        for l in self.convs.values():
            x = l(x)
        return x
      
model = Model()
model

Model(
  (convs): ModuleDict(
    (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  )
)

## 自動微分機能

In [0]:
# Tensorを作る
x = torch.tensor(1, requires_grad=False, dtype=torch.float32)
w = torch.tensor(2, requires_grad=True, dtype=torch.float32)
b = torch.tensor(3, requires_grad=True, dtype=torch.float32)

# 計算グラフを作成する
y = w * x + b    # y = 1 * 2 + 3

# Tensorを返す
print("y：", y)

# 勾配計算
y.backward()

# 勾配を確認
print("x.grad：", x.grad) #requires_grad=Falseなので微分計算がない。
print("w.grad：", w.grad)
print("b.grad：", b.grad)

y： tensor(5., grad_fn=<ThAddBackward>)
x.grad： None
w.grad： tensor(1.)
b.grad： tensor(1.)


In [0]:
#ネットワークのパラメータを凍結する
net = torchvision.models.vgg16(pretrained=True)
for param in net.features.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:25<00:00, 22114532.84it/s]


## GPUを使用する

- PyTorch 0.4.0以前

In [10]:
xf_gpu = torch.FloatTensor(1, 2).cuda()
xf_gpu

tensor([[509714.,      0.]], device='cuda:0')

In [11]:
xf_cpu = xf_gpu.cpu()
xf_cpu

tensor([[509714.,      0.]])

In [12]:
xf_numpy = xf_cpu.numpy()
xf_numpy

array([[509714.,      0.]], dtype=float32)

- PyTorch 0.4.0, PyTorch 0.4.1

In [15]:
xf_gpu = torch.FloatTensor(1, 2).to('cuda')
xf_gpu

tensor([[5.0971e+05, 0.0000e+00]], device='cuda:0')

In [16]:
xf_cpu = xf_gpu.to('cpu')
xf_cpu

tensor([[5.0971e+05, 0.0000e+00]])

- PyTorch 0.4.0以前

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))
net = Model()
net.cuda()

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
)

- PyTorch 0.4.0, PyTorch 0.4.1

In [17]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))
net = Model()
net.to('cuda')

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
)